In [ ]:
WORKDIR = "path/to/WORKDIR/"  # specify WORKDIR here

In [ ]:
import matplotlib.pyplot as plt
from src.paper_utils import get_figsize, configure_matplotlib

In [ ]:
configure_matplotlib(
    rc={
        "xtick.labelbottom": False,
        "xtick.bottom": True,
        "xtick.labeltop": True,
        "ytick.left": True,
        "figure.constrained_layout.use": False,
        "savefig.pad_inches": 0.01,
    }
)

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
from tqdm import tqdm

In [ ]:
experiments = [ "gragnaniello2021/progan", "gragnaniello2021/stylegan2",
                "wang2020/blur_jpg_prob0.1", "wang2020/blur_jpg_prob0.5",
                "wang2020/finetuning_All", "wang2020/finetuning_GAN", "wang2020/finetuning_DM",
                "wang2020/scratch_All", "wang2020/scratch_GAN", "wang2020/scratch_DM"]
n_total = 10000       # use all 10k images

In [ ]:
for experiment in tqdm(experiments):
    base_input_folder = f"{WORKDIR}/output/features/{experiment}"

    # load feature representations
    X_real = np.load(base_input_folder + f'/features_Real.npy')[:n_total]

    X_DDPM = np.load(base_input_folder + f'/features_DDPM.npy')[:n_total]
    X_IDDPM = np.load(base_input_folder + f'/features_IDDPM.npy')[:n_total]
    X_ADM = np.load(base_input_folder + f'/features_ADM.npy')[:n_total]
    X_PNDM = np.load(base_input_folder + f'/features_PNDM.npy')[:n_total]
    X_LDM = np.load(base_input_folder + f'/features_LDM.npy')[:n_total]

    X_ProGAN = np.load(base_input_folder + f'/features_ProGAN.npy')[:n_total]
    X_StyleGAN = np.load(base_input_folder + f'/features_StyleGAN.npy')[:n_total]
    X_ProjectedGAN = np.load(base_input_folder + f'/features_ProjectedGAN.npy')[:n_total]
    X_DiffStyleGAN2 = np.load(base_input_folder + f'/features_Diff-StyleGAN2.npy')[:n_total]
    X_DiffProjectedGAN = np.load(base_input_folder + f'/features_Diff-ProjectedGAN.npy')[:n_total]

    # concatenate all representations
    Xs = np.concatenate([X_real, X_DDPM, X_IDDPM, X_ADM, X_PNDM, X_LDM,
                     X_ProGAN, X_StyleGAN, X_ProjectedGAN, X_DiffStyleGAN2, X_DiffProjectedGAN])

    Xs_embedded = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=3, verbose=True).fit_transform(Xs)
    # save embedding
    np.save(base_input_folder + "/Xs_embedded.npy", Xs_embedded)

In [ ]:
# Visualize the t-SNE embeddings

cm = plt.get_cmap('coolwarm')

size = 1.5 # size of each marker
n_plot = 250  # number of samples to visualize per class
alpha = 0.4   # opacity

fig, axes = plt.subplot_mosaic(
    """
    0123
    456.
    789.
    """,
    # set the height ratios between the rows
    height_ratios=[1, 1, 1],
    # set the width ratios between the columns
    width_ratios=[1, 1, 1, 1],
    constrained_layout=False,
    figsize=get_figsize(ratio=0.52),
    sharey=False,
    sharex=False
)

titles = [  'Gragnaniello2021' + r'\vspace{-0.5em} \tiny{(ProGAN)}',
            'Gragnaniello2021' + r'\vspace{-0.5em} \tiny{(StyleGAN2)}',
            'Wang2020' + r'\vspace{-0.5em} \tiny{(Blur+JPEG (0.1))}',
            'Wang2020' + r'\vspace{-0.5em} \tiny{(Blur+JPEG (0.5))}',
            'Wang2020' + r'\vspace{-0.5em} \tiny{(fine-tuned on \textbf{All})}',
            'Wang2020' + r'\vspace{-0.5em} \tiny{(fine-tuned on \textbf{GANs})}',
            'Wang2020' + r'\vspace{-0.5em} \tiny{(fine-tuned on \textbf{DMs})}',
            'Wang2020' + r'\vspace{-0.5em} \tiny{(trained on \textbf{All})}',
            'Wang2020' + r'\vspace{-0.5em} \tiny{(trained on \textbf{GANs})}',
            'Wang2020' + r'\vspace{-0.5em} \tiny{(trained on \textbf{DMs})}'    ]

for i, experiment in enumerate(experiments):

    ax = axes[str(i)]

    base_input_folder = f"{WORKDIR}/output/features/{experiment}"

    # load embedded features and extract per model
    Xs_embedded = np.load(base_input_folder + "/Xs_embedded.npy")

    X_real_embedded = Xs_embedded[:n_plot]
    X_DDPM_embedded = Xs_embedded[n_total:(n_total + n_plot)]
    X_IDDPM_embedded = Xs_embedded[2*n_total:(2*n_total + n_plot)]
    X_ADM_embedded = Xs_embedded[3*n_total:(3*n_total + n_plot)]
    X_PNDM_embedded = Xs_embedded[4*n_total:(4*n_total + n_plot)]
    X_LDM_embedded = Xs_embedded[5*n_total:(5*n_total + n_plot)]

    X_ProGAN_embedded = Xs_embedded[6*n_total:(6*n_total + n_plot)]
    X_StyleGAN_embedded = Xs_embedded[7*n_total:(7*n_total + n_plot)]
    X_ProjectedGAN_embedded = Xs_embedded[8*n_total:(8*n_total + n_plot)]
    X_DiffStyleGAN2_embedded = Xs_embedded[9*n_total:(9*n_total + n_plot)]
    X_DiffProjectedGAN_embedded = Xs_embedded[10*n_total:(10*n_total + n_plot)]

    ax.set_title(titles[i])

    # plot DMs
    ax.scatter(X_DDPM_embedded[:n_plot,0], X_DDPM_embedded[:n_plot,1], alpha = alpha, s=size, label="DDPM", color=cm(1.0), zorder=4)
    ax.scatter(X_IDDPM_embedded[:n_plot,0], X_IDDPM_embedded[:n_plot,1], alpha = alpha, s=size, label="IDDPM", color=cm(0.9),zorder=4)
    ax.scatter(X_ADM_embedded[:n_plot,0], X_ADM_embedded[:n_plot,1], alpha = alpha, s=size, label="ADM", color=cm(0.8),zorder=4)
    ax.scatter(X_PNDM_embedded[:n_plot,0], X_PNDM_embedded[:n_plot,1], alpha = alpha, s=size, label="PNDM", color=cm(0.7),zorder=4)
    ax.scatter(X_LDM_embedded[:n_plot,0], X_LDM_embedded[:n_plot,1], alpha = alpha, s=size, label="LDM", color=cm(0.6),zorder=4)

    # plot GANs
    ax.scatter(X_ProGAN_embedded[:n_plot,0], X_ProGAN_embedded[:n_plot,1], alpha = alpha, s=size, label="ProGAN", color=cm(0),zorder=4)
    ax.scatter(X_StyleGAN_embedded[:n_plot,0], X_StyleGAN_embedded[:n_plot,1], alpha = alpha, s=size, label="StyleGAN", color=cm(0.1),zorder=4)
    ax.scatter(X_ProjectedGAN_embedded[:n_plot,0], X_ProjectedGAN_embedded[:n_plot,1], alpha = alpha, s=size, label="ProjectedGAN", color=cm(0.2),zorder=4)
    ax.scatter(X_DiffStyleGAN2_embedded[:n_plot,0], X_DiffStyleGAN2_embedded[:n_plot,1], alpha = alpha, s=size, label="Diff-StyleGAN2", color=cm(0.3),zorder=4)
    ax.scatter(X_DiffProjectedGAN_embedded[:n_plot,0], X_DiffProjectedGAN_embedded[:n_plot,1], alpha = alpha, s=size, label="Diff-ProjectedGAN", color=cm(0.4),zorder=4)

    # plot reals
    ax.scatter(X_real_embedded[:n_plot,0], X_real_embedded[:n_plot,1], alpha = alpha, s=size, label="Real", c="g",zorder=4)

handles, labels = fig.gca().get_legend_handles_labels()

by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), markerscale=4, ncol=1, borderpad=0.7,
           bbox_to_anchor=(0.22, 0.495), labelspacing = 0.8, columnspacing=0.8)
# plt.savefig(f"TSNE.pdf")
plt.show()